In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

from tqdm import tqdm

import torch

from utils.data import get_hsm_dataset, split_data
from utils.metrics import MAPE, WAPE, MAE
from utils.dl import QuantGAN_Discriminator, QuantGAN_Generator
from utils.QuantGAN_gaussianize import Gaussianize

In [2]:
dataset_path = "data/huge_stock_market_dataset/"
synthetic_path = f"{dataset_path}synthetic/QuantGAN/"
models_dir = "models/"

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

val_size = 0.0
test_size = 0.3
batches_to_gen = 20

num_epochs = 100
nz = 3
batch_size = 80
seq_len = 127
clip = 0.01
lr = 0.0002
receptive_field_size = 127

cuda:0


In [4]:
class Loader32(torch.utils.data.Dataset):
    def __init__(self, data, length):
        assert len(data) >= length
        self.data = data
        self.length = length
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx:idx+self.length]).reshape(- 1, self.length).to(torch.float32)
        
    def __len__(self):
        return max(len(self.data)-self.length, 0)

In [5]:
def model_routine(time_series, ts_index):
    global val_size, test_size, batches_to_gen, num_epochs, nz, batch_size, seq_len, clip, lr, receptive_field_size
    #  using only train set to generate new samples
    (train_ts, *_), *_ = split_data(time_series, val_size=val_size, test_size=test_size)
    
    # preprocessing steps according to the QuanGAN paper
    df = train_ts
    returns = df.shift(1) / df - 1
    log_returns = np.log(df / df.shift(1))[1:].to_numpy().reshape(- 1, 1)
    standardScaler1 = StandardScaler()
    standardScaler2 = StandardScaler()
    gaussianize = Gaussianize()
    log_returns_preprocessed = standardScaler2.fit_transform(gaussianize.fit_transform(standardScaler1.fit_transform(log_returns)))
    data_size = log_returns.shape[0]

    # defining models and optimizers
    generator = QuantGAN_Generator().to(device)
    discriminator = QuantGAN_Discriminator(seq_len).to(device)
    disc_optimizer = torch.optim.RMSprop(discriminator.parameters(), lr=lr)
    gen_optimizer = torch.optim.RMSprop(generator.parameters(), lr=lr)

    # data preparing
    dataset = Loader32(log_returns_preprocessed, 127)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
    
    t = tqdm(range(num_epochs))
    for epoch in t:
        for idx, data in enumerate(dataloader, 0):

            discriminator.zero_grad()
            real = data.to(device)
            batch_size, seq_len = real.size(0), real.size(2)
            noise = torch.randn(batch_size, nz, seq_len, device=device)
            fake = generator(noise).detach()
            disc_loss = - torch.mean(discriminator(real)) + torch.mean(discriminator(fake))
            disc_loss.backward()
            disc_optimizer.step()

            for dp in discriminator.parameters():
                dp.data.clamp_(-clip, clip)
    
            if idx % 5 == 0:
                generator.zero_grad()
                gen_loss = - torch.mean(discriminator(generator(noise)))
                gen_loss.backward()
                gen_optimizer.step()
        t.set_description('Discriminator Loss: %.8f Generator Loss: %.8f' % (disc_loss.item(), gen_loss.item()))
    # saving model
    torch.save(generator, models_dir +  f'QuantGAN_generator_selected{ts_index}.pth')

    # generation synthetic time series
    generator.eval()
    ys = []
    for _ in range(batches_to_gen):
        with torch.no_grad():
            noise = torch.randn(80, 3, 127).to(device)
            y = generator(noise).cpu().detach().squeeze()

        y = (y - y.mean(axis=0)) / y.std(axis=0)
        y = standardScaler2.inverse_transform(y)
        y = np.array([gaussianize.inverse_transform(np.expand_dims(x, 1)) for x in y]).squeeze()
        y = standardScaler1.inverse_transform(y)

        # some basic filtering to redue the tendency of GAN to produce extreme returns
        y = y[(y.max(axis=1) <= 2 * log_returns.max()) & (y.min(axis=1) >= 2 * log_returns.min())]
        y -= y.mean()
        ys.append(y)

    np.save(synthetic_path + f"selected{ts_index}.npy", np.row_stack(ys))

    del discriminator, generator, disc_loss, gen_loss, dataloader, dataset, y
    torch.cuda.empty_cache()

In [6]:
ts_iterator = get_hsm_dataset(dataset_path, selected_files=f"{dataset_path}/selected.csv")
start_point = 22
for _ in range(start_point): next(ts_iterator)

for ts_index, time_series in enumerate(ts_iterator, start_point):
    print(f"Time Series #{ts_index}")
    model_routine(time_series, ts_index)

Time Series #22


Discriminator Loss: 0.00000012 Generator Loss: -0.50113839: 100%|██████████| 100/100 [05:19<00:00,  3.20s/it]


Time Series #23


Discriminator Loss: 0.00000021 Generator Loss: -0.49835110: 100%|██████████| 100/100 [04:44<00:00,  2.85s/it]


# Similarity

In [3]:
from tqdm import tqdm
from pathlib import Path

import numpy as np
import pandas as pd
from scipy.special import kl_div

from utils.data import get_hsm_dataset, split_data, log_returns

In [4]:
dataset_path = Path("data/huge_stock_market_dataset/")
results_dir = Path("results")

val_size = 0.0
test_size = 0.3
seq_len = 127

sj_div = lambda x, y: (kl_div(x, (x + y) / 2) + kl_div(y, (x + y) / 2)) / 2
min_max_norm = lambda x: (x - x.min()) / (x.max() - x.min())

In [8]:
for model in ("QuantGAN",):
    synthetic_path = dataset_path / f"synthetic/{model}/"
    results = {"kl_div": [], "sj_div": []}
    ts_iterator = get_hsm_dataset(dataset_path, selected_files=f"{dataset_path}/selected.csv")

    for ts_index, time_series in tqdm(enumerate(ts_iterator), leave=False):
        (train_ts, *_), *_ = split_data(time_series, val_size=val_size, test_size=test_size)
        train_ts = log_returns(train_ts).values.flatten()
        train_ts = min_max_norm(train_ts)
        train_tss = [train_ts[i: i + seq_len] for i in range(len(train_ts) - seq_len + 1)]
        
        synth_tss = np.load(synthetic_path / f"selected{ts_index}.npy")
        kl_div_res = sj_div_res = 0
        for synth_ts in tqdm(synth_tss):
            synth_ts = min_max_norm(synth_ts)
            for train_ts in train_tss:
                res = kl_div(synth_ts, train_ts)
                kl_div_res += np.where(np.isinf(res), 0, res).mean()
                sj_div_res += sj_div(synth_ts, train_ts).mean()
        results["kl_div"].append(kl_div_res / len(synth_tss) / len(train_tss))
        results["sj_div"].append(sj_div_res / len(synth_tss) / len(train_tss))
    
    pd.DataFrame(results).to_csv(results_dir / f"synth_sim_{model}.csv", index=False)

100%|██████████| 1599/1599 [03:52<00:00,  6.87it/s]
